In [ ]:
import os
import requests
import json
import psycopg2
from psycopg2 import sql

# Función para conectarse a la base de datos de Redshift
def conectar_redshift():
    print("Conectando a Redshift...")
    try:
        conn = psycopg2.connect(
            dbname=os.environ['REDSHIFT_DBNAME'],
            user=os.environ['REDSHIFT_USER'],
            password=os.environ['REDSHIFT_PASSWORD'],
            host=os.environ['REDSHIFT_HOST'],
            port=os.environ['REDSHIFT_PORT']
        )
        print("Conexión exitosa.")
        return conn
    except Exception as e:
        print("Error al conectar a Redshift:", e)
        return None

In [ ]:
# Función para crear una tabla en Redshift
def crear_tabla_redshift(conn):
    cursor = conn.cursor()
    create_table_query = """
    CREATE TABLE IF NOT EXISTS data_criptomonedas (
        id VARCHAR(255) PRIMARY KEY,
        nombre VARCHAR(100),
        simbolo VARCHAR(10),
        precio FLOAT,
        market_cap_rank INTEGER,
        last_updated TIMESTAMP,
        fecha_ingesta TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()

def extraer_cargar_datos(conn):
    # Hacer una solicitud a la API de CoinGecko para obtener datos de criptomonedas
    response = requests.get('https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd')
    data = response.json()

    # Conectar a Redshift y crear tabla si no existe
    cursor = conn.cursor()

    # Insertar datos en la tabla
    for item in data:
        id_api_valor = item['id']
        nombre = item['name']
        simbolo = item['symbol']
        precio = item['current_price']
        market_cap_rank = item['market_cap_rank']
        last_updated = item['last_updated']
        cursor.execute("INSERT INTO data_criptomonedas (id, nombre, simbolo, precio, market_cap_rank, last_updated) VALUES (%s, %s, %s, %s, %s, %s)", (id_api_valor, nombre, simbolo, precio, market_cap_rank, last_updated))
    conn.commit()
    cursor.close()

# Ejecutar el script
conn = conectar_redshift()
if conn:
    crear_tabla_redshift(conn)
    extraer_cargar_datos(conn)
else:
    print("No se pudo conectar a Redshift.")




Conectando a Redshift...
Conexión exitosa.
